In [7]:
import pandas as pd

# Specifying the weights per location category

In [8]:
marks = {"vegan_venues": 0.20, "starbucks":0.20, "schools":0.20, "train_stations":0.25, "basketball_courts":0.15} 

# Loading the dataframes from the Mongo geoqueries

In [75]:
dfsanfran = pd.read_csv("../newdata/dfgeosf.csv")
dfell = pd.read_csv('../newdata/dfgeoell.csv')
dfbro = pd.read_csv('../newdata/dfgeobro.csv')

# Making a new column with the punctuation per location (weights*distance)

In [76]:
def punct(df):  
    o = []
    for i,row in df.iterrows():
        if row['category'] == "vegan_venues":
            o.append(row['geodist']*marks['vegan_venues'])

        elif row['category'] == "starbucks":
            o.append(row['geodist']*marks['starbucks'])

        elif row['category'] == "schools":
            o.append(row['geodist']*marks['schools'])

        elif row['category'] == "train_stations":
            o.append(row['geodist']*marks['train_stations'])

        elif row['category'] == "basketball_courts":
            o.append(row['geodist']*marks['basketball_courts'])
    df['punctuation'] = o
    return df['punctuation']

In [77]:
punct(dfsanfran)
punct(dfell)
punct(dfbro)

0     0.037679
1     0.037680
2     0.030144
3     0.030144
4     0.030144
        ...   
91    0.030150
92    0.030151
93    0.030153
94    0.037691
95    0.030153
Name: punctuation, Length: 96, dtype: float64

In [78]:
dfsanfran

,name,lat,lon,category,geodist,punctuation
0,Victoria Manalo Draves Basketball Court,37.777412,-122.406676,basketball_courts,0.152311,0.022847
1,Bessie Carmichael Elementary School,37.776266,-122.406351,schools,0.255062,0.051012
2,Ukrainian School Of San Francisco,37.776299,-122.406776,schools,0.264012,0.052802
3,John Scott School Of Voice,37.781410,-122.407547,schools,0.368342,0.073668
4,Old School Records,37.781863,-122.406024,schools,0.378114,0.075623
...,...,...,...,...,...,...
73,Ida B. Wells High School,37.775287,-122.434482,schools,2.563845,0.512769
74,Everett Middle School,37.763651,-122.428794,schools,2.622618,0.524524
75,Children's Day School (Middle School) Extension,37.759659,-122.425600,schools,2.733690,0.546738
76,Sanchez Elementary School,37.763723,-122.430825,schools,2.758956,0.551791


# Normalizing the data

## San Francisco

In [79]:
def normalization(df, col):
    df['normal'] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())     
    df = df.reset_index(drop = False)
    return df

In [80]:
pundfsanfran = normalization(dfsanfran,'geodist')
pundfsanfran = dfsanfran.groupby('category').agg({'normal':'mean'}).reset_index()

In [81]:
pundfsanfran

,category,normal
0,basketball_courts,0.294989
1,schools,0.401540
2,starbucks,0.314408
3,train_stations,0.438440
4,vegan_venues,0.295980


In [82]:
meansanfran = sum([i for i in pundfsanfran['normal']])/len(pundfsanfran['normal'])
meansanfran

0.34907135567294617

## Ellensburg

In [83]:
pundfell = normalization(dfell,'geodist')
pundfell = dfell.groupby('category').agg({'normal':'mean'}).reset_index()

In [84]:
pundfell

,category,normal
0,basketball_courts,0.548220
1,schools,0.244937
2,starbucks,0.265862
3,train_stations,0.343203


In [85]:
meanell = sum([i for i in pundfell['normal']])/len(pundfell['normal'])
meanell

0.35055563387786254

## Brooklyn

In [86]:
pundfbro = normalization(dfbro,'geodist')
pundfbro = dfbro.groupby('category').agg({'normal':'mean'}).reset_index()

In [87]:
pundfbro

,category,normal
0,basketball_courts,0.388479
1,schools,0.389627
2,starbucks,0.468343
3,train_stations,0.330886
4,vegan_venues,0.400825


In [88]:
meanbrooklyn = sum([i for i in pundfbro['normal']])/len(pundfbro['normal'])
meanbrooklyn

0.39563203410156744

# Calculating the mean values of the different cities

In [89]:
means = [meansanfran, meanell, meanbrooklyn]
col = ['San Francisco', 'Ellensburg', 'Brooklyn']
finalmeans = pd.DataFrame(means, index = col).T
finalmeans

,San Francisco,Ellensburg,Brooklyn
0,0.349071,0.350556,0.395632


 <font size="5"> Taking into account the studies, the best city to start our new company is San Francisco, followed by Brooklyn (as Ellensburg doesn't have any vegan restaurant and that is one of the requirements).</font> 

In [90]:
finalmeans.to_csv("../newdata/finalmeans.csv")